<a href="https://colab.research.google.com/github/KseniyaBarinova/nlp_course/blob/Barinova_Exam/%22Exam_task_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers accelerate evaluate scikit-learn

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from sklearn.metrics import classification_report

from transformers import BertTokenizer, BertForSequenceClassification
import evaluate
import torch
from torch.utils.data import DataLoader
from transformers import TextClassificationPipeline
from datasets import load_dataset

In [ ]:
# Загрузка предобученного BERT
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Загрузка датасета
dataset = load_dataset("glue", "sst2")
train_dataset = dataset["train"]
test_dataset = dataset["validation"]

In [ ]:
# Предобработка данных
def preprocess(example):
    return tokenizer(example["sentence"], padding="max_length", truncation=True, return_tensors="pt")


In [ ]:
train_dataset = train_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

In [ ]:
# Создание DataLoader'ов
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

In [ ]:
# Функция для оценки качества модели
def evaluate(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in data_loader:
            inputs = {key: val.squeeze(1) for key, val in batch.items()}
            labels = inputs.pop("labels")
            outputs = model(**inputs)
            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total


In [ ]:
# Оценка качества на тестовом датасете
test_accuracy = evaluate(model, test_loader)
print("Test Accuracy:", test_accuracy)

AttributeError: 'list' object has no attribute 'squeeze'

In [ ]:
# Загрузка модели и токенизатора
# model_name = "bert-base-uncased"  # Замените на нужное имя модели
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)


tokenizer = AutoTokenizer.from_pretrained("sileod/deberta-v3-base-tasksource-nli")
model = AutoModelForSequenceClassification.from_pretrained("sileod/deberta-v3-base-tasksource-nli")

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/18.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

In [ ]:
# Загрузка датасета
dataset = load_dataset("imdb")  # Замените на нужный датасет

In [ ]:
# Токенизация и подготовка данных
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

dataset = dataset.map(tokenize, batched=True)
dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# Предсказание меток
predictions = []
labels = []

for i in range(len(dataset)):
    input_ids = dataset['input_ids'][i].unsqueeze(0)
    attention_mask = dataset['attention_mask'][i].unsqueeze(0)
    label = dataset['label'][i].item()
    labels.append(label)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions.append(outputs.logits.argmax().item())


KeyError: 'input_ids'

In [ ]:
# Подсчет метрик
print(classification_report(labels, predictions))

Скрипт выполняет классификацию текста с помощью предобученной модели на заданном датасете и выводит отчет о классификации с метриками, такими как точность, полнота и F1-мера